## A simple swarm

In [9]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from core.agent import *
from core.world import * 
import numpy as np
from core.render import * 

In [11]:
world = World((100, 100))
swarm = [Agent() for _ in range(1000)]
for agent in swarm:
    world.add_agent(agent)

In [12]:
swarm = initialize_positions_randomly(world, swarm)

In [13]:
# Test movements by doing random actions
def update():
    for agent in swarm:
        choice = np.random.randint(1, 5)
        agent.move(choice)

    world.update()

update()

This ran


In [14]:
render_world(world, (800, 800), update_fn=update)

This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
This ran
